In [1]:
# Get DataFrame
from src.tdc_constant import TDC
from src.tdc_data import get_data_df, get_mtl_data_df

get_data_df(TDC.Clearance, datasetType = 'train')
get_mtl_data_df([TDC.Clearance, TDC.BBB], datasetType = 'train', scaled=True)


Drug  BBB  Clearance
0                CC(C)(C)OC(=O)CCCc1ccc(N(CCCl)CCCl)cc1  1.0        NaN
1                CC(C)(C)OC(=O)CCCc1ccc(N(CCCl)CCCl)cc1  1.0        NaN
2      CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23  1.0        NaN
3      CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23  1.0        NaN
4     Cc1onc(-c2ccccc2Cl)c1C(=O)N[C@@H]1C(=O)N2[C@@H...  1.0        NaN
...                                                 ...  ...        ...
2220            CCc1cccc2c3c([nH]c12)C(CC)(CC(=O)O)OCC3  NaN  -0.601052
2221  Cc1cc(CN2Cc3ccccc3C2C(=O)Nc2ccc(Cl)cc2Cl)ccc1O...  NaN  -0.558425
2222  COc1cccc2c1c(NS(=O)(=O)c1ccc(Cl)s1)nn2Cc1cccc(...  NaN  -0.457888
2223  CO[C@H]1CC[C@]2(CC1)Cc1ccc(-c3cc(Cl)cc(C#N)c3)...  NaN  -0.606883
2224  COc1ccc(COc2nc(Br)cnc2NS(=O)(=O)c2ccc(C)cc2)cc...  NaN  -0.236706

[2225 rows x 3 columns]

In [ ]:
from src.tdc_constant import TDC
from src.tdc_data import get_scaler

clearance_scaler = get_scaler(TDC.Clearance)
solubility_scaler = get_scaler(TDC.Solubility)

from joblib import dump, load

dump(clearance_scaler, "clearance_scaler.bin", compress=True)
dump(solubility_scaler, "solubility_scaler.bin", compress=True)

sc = load("solubility_scaler.bin")
print(sc.mean_)

## Get Embedding Vector

In [6]:
# Get embedding vector
model_name = 'MTL2_DropRatio0.2/MolCLR_[BBB, CYP3A4, Clearance, Solubility]_sc-12.13_1830.pt'
modelf=f'ckpts/{model_name}'

device = 'cuda:0'

In [7]:
from src.ginet_finetune import GINet_Feat, GINet_Feat_MTL, load_pre_trained_weights

best_model = GINet_Feat(
    pool="mean",
    drop_ratio=0,
).to(device)
best_model = load_pre_trained_weights(best_model, "gin.", device, modelf).to(device)

# Try several times to check loading correctly
print(best_model.gnns[0].edge_embedding1.weight[:5])
# print(best_model.mtl.pred_heads[0][0].weight[:5])

Loaded pre-trained model with success.


In [15]:
import torch
import numpy as np
import pandas as pd
from src.dataset_mtl import smilesToGeometric

drug_dataset = (
    "/data/project/aigenintern/2023-2/DCC/DrugMAP_approved_smallmolecule_drug.csv"
)
drug_df = pd.read_csv(drug_dataset)

comp_dataset = "/data/project/aigenintern/2023-2/DCC/ZINC_compound.csv"
comp_df = pd.read_csv(comp_dataset)

df = pd.DataFrame()
df.index.name = "SMILES"
df["isDrug"] = pd.Series(dtype="bool")

embedding_vector_list = []
smiles_list = []
isDrug_list = []

best_model.eval()


def convertSmiles(lst, isDrug):
    for smiles in lst:
        data = smilesToGeometric(smiles)
        data.batch = torch.tensor([0] * len(data.x))
        embedding_vector = (
            best_model(data.to(device)).flatten().flatten().detach().cpu().numpy()
        )

        embedding_vector_list.append(embedding_vector.tolist())
        smiles_list.append(smiles)
        isDrug_list.append(isDrug)


convertSmiles(drug_df["rdkit_canonical_smiles"].values, True)
convertSmiles(comp_df["rdkit_canonical_smiles"].values, False)

print(len(smiles_list))

df = pd.DataFrame(np.array(embedding_vector_list))
df = df.assign(SMILES=smiles_list)
df = df.assign(isDrug=isDrug_list)
df.set_index("SMILES", inplace=True)

df.to_csv("chem_space.csv")

/tmp/ipykernel_20812/2973276160.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[smiles, "1"] = "1"
